# Agent For Financial Analysis

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from utils import get_openai_api_key, get_serper_api_key
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

import os


c:\Users\sarth\crewai\.conda\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\sarth\crewai\.conda\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
c:\Users\sarth\crewai\.conda\Lib\site-packages\crewai_tools\tools\selenium_scraping_tool\selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic 

In [3]:
load_dotenv()

os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo-0125'
os.environ['SERPER_API_KEY'] = get_serper_api_key()
os.environ['OPENAI_API_KEY'] = get_openai_api_key()


In [4]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)


In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)


In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'PYPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}


In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: daf287fe-711b-474b-bf22-992e560c56be                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 435ef155-af47-4658-83bd-8046d70f7986
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 435ef155-af47-4658-83bd-8046d70f7986
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (PYPL). Use statistical modeling and machine learning to identify trends and predict market movements.


🚀 Crew: crew
└── 📋 Task: 435ef155-af47-4658-83bd-8046d70f7986
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Crew Manager
## Final Answer: 
Below is a comprehensive risk analysis report detailing potential risks and mitigation recommendations for PYPL based on the information gathered:

Risk Analysis Report for PYPL:

1. Market Volatility: The recent downgrade of PYPL from 'Buy' to 'Neutral' by Arete has led to a pre-market trading decline of approximately -1%. This indicates increased market volatility and uncertainty surrounding the stock.

Mitigation Strategy: Develop a risk management plan that includes setting stop-loss orders and closely monitoring market trends to react quickly to any significant price movements.

2. Company Performance: The downgrade suggests that Arete has concerns about PYPL's future performance, which could impact investor confidence and stock price stability.

Mitigation Strategy: Conduct a thorough analysis of PYPL's financial health, business strategies, and competitive position to assess the company's long-term growth prospects and mitigate potential

🚀 Crew: crew
├── 📋 Task: 435ef155-af47-4658-83bd-8046d70f7986
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   └── 🔧 Using Ask question to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Used Read website content (1)
├── 📋 Task: ab8e2265-d8a1-4af7-8013-f7eda67bb2c7
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   ├── 🔧 Used Read website content (2)
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (3)
├── 📋 Task: f5a58a93-b523-439d-9cea-6d174a7c741f
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   ├── 🔧 Used Read website content (3)
│   │   ├── 🔧 Used Delegate work to coworker (3)
│   │   └── 🔧 Using Ask question to coworker (2)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (4)
│       └── 🔧 Used Read website content (4)
└── 📋 Task: 514d2b94-1058-41d3-beb2-057da601aec1
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   ├── 🔧 Used Read website content (5)
    │   └── 🔧 Using Delegate work to coworker (4)
    ├── 🤖 Agent: Risk Advisor
    │       Status: ✅ Completed
    │   ├── 🔧 Used Search the internet with Serper (5)
    │   ├── 🔧 Used Read website content (6)
    │   └── 🔧 Using Ask question to coworker (3)
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed
        ├── 🔧 Used Search the internet with Serper (7)
        └── 🔧 Used Read website content (8)

🚀 Crew: crew
├── 📋 Task: 435ef155-af47-4658-83bd-8046d70f7986
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   └── 🔧 Using Ask question to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Used Read website content (1)
├── 📋 Task: ab8e2265-d8a1-4af7-8013-f7eda67bb2c7
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   ├── 🔧 Used Read website content (2)
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (3)
├── 📋 Task: f5a58a93-b523-439d-9cea-6d174a7c741f
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   ├── 🔧 Used Read website content (3)
│   │   ├── 🔧 Used Delegate work to coworker (3)
│   │   └── 🔧 Using Ask question to coworker (2)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (4)
│       └── 🔧 Used Read website content (4)
└── 📋 Task: 514d2b94-1058-41d3-beb2-057da601aec1
       Assigned to: Crew Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   ├── 🔧 Used Read website content (5)
    │   └── 🔧 Using Delegate work to coworker (4)
    ├── 🤖 Agent: Risk Advisor
    │       Status: ✅ Completed
    │   ├── 🔧 Used Search the internet with Serper (5)
    │   ├── 🔧 Used Read website content (6)
    │   └── 🔧 Using Ask question to coworker (3)
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed
        ├── 🔧 Used Search the internet with Serper (7)
        └── 🔧 Used Read website content (8)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 514d2b94-1058-41d3-beb2-057da601aec1                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: daf287fe-711b-474b-bf22-992e560c56be                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
print(result)


Below is a comprehensive risk analysis report detailing potential risks and mitigation recommendations for PYPL based on the information gathered:

Risk Analysis Report for PYPL:

1. Market Volatility: The recent downgrade of PYPL from 'Buy' to 'Neutral' by Arete has led to a pre-market trading decline of approximately -1%. This indicates increased market volatility and uncertainty surrounding the stock.

Mitigation Strategy: Develop a risk management plan that includes setting stop-loss orders and closely monitoring market trends to react quickly to any significant price movements.

2. Company Performance: The downgrade suggests that Arete has concerns about PYPL's future performance, which could impact investor confidence and stock price stability.

Mitigation Strategy: Conduct a thorough analysis of PYPL's financial health, business strategies, and competitive position to assess the company's long-term growth prospects and mitigate potential risks.

3. External Influences: External 